## Notebook for 1D BDT scan and calculating FOMs in Bd2psi2SKS on data


In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/home/chasenberg/repos/')
import os
import sys
import ROOT
from ROOT import TFile, TH1D, TH2D, TCanvas, gStyle, TLine, TTree
from ROOT import (RooArgSet, RooRealVar, RooDataSet, RooPlot, RooFit, RooStats, RooArgList)
import root_pandas as rp
import root_numpy as ry 

import math
import itertools
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

from root_pandas import read_root
import pandas as pd

sys.path.append('/home/vmueller/repos')
from dopy.doroot.root_utils import read_roodataset_from_tree, plot_pulls
from dopy.doplot.plotting import Plotter, Plot                

In [2]:

from ROOT import RooRealVar, RooExponential, RooExtendPdf, RooAddPdf, RooArgList, RooGaussian
from ROOT import RooFormulaVar
from ROOT import gSystem
gSystem.Load('libRooFit.so')
gSystem.Load('/home/chasenberg/repos/dopy/dolib/CustomShapes/libKll')
from ROOT import RooIpatia2

### Read ROOT file

In [3]:
dataset_file_name = '/fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks/Bd2JpsiKS_pv_selected.root'

In [4]:
branches = [
            'B0_TAGOMEGA_OS',
            'B0_TAGDECISION_OS',
            'B0_TAU',
            'B0_TAUERR',
            'B0_TRUETAU',
            'B0_FitPVConst_MinIPCHI2anyPV_flat',
            'idxRunNumber',
            'idxEventNumber',
            'idxPV',
            'BDTresponse_wrongPV',
            'Delta_TAU'
            ]

In [5]:
tree_mc = 'Bd2JpsiKS'
signal_dataframe_wrongPV  = rp.read_root(dataset_file_name,key=tree_mc,columns=branches, flatten=False)
signal_dataframe_wrongPV  = signal_dataframe_wrongPV.replace([np.inf, -np.inf], np.nan)
signal_dataframe_wrongPV  = signal_dataframe_wrongPV.dropna()

In [6]:
signal_dataframe_wrongPV.to_root('/fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks/interim_bdtapproach.root',key='Bd2JpsiKS')

### Function to calculate the FOMs

### Configure  and define cuts 

In [7]:
lowerbound=5220 # rather arbitrary signal window definition
upperbound=5450
#B0_FitDaughtersConst_M.setRange('SIGREGION',lowerbound,upperbound)

subdir = '/home/chasenberg/plots/wrongpv_cuttuning'
plot_dir    = os.path.join(subdir, 'plots')
results_dir = os.path.join(subdir, 'results')
plot_file   = os.path.join(plot_dir, 'all_plots.pdf')


if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)
if not os.path.exists(results_dir):
    os.makedirs(results_dir)
    
mass_var_name = 'B0_FitDaughtersConst_M'
cuttype = 'Delta_TAU'#'B0_TAUERR'#

cuts = np.linspace(0.00001,0.01,100) #(0,20,10)    #(0,0.001,10)
#cuts = [-0.05]
cuts_for_which_to_store_sdata = []   

num_sig = []
num_bkg = []
num_all = []

cut_to_performance = {} 
cut_to_shape = {}

perform_fit = True
do_splot    = True
postfix = ''
final_params_file = None
oldParameters = None
final_model = None

### Calculate FOM

In [ ]:
def calculate_foms(data_fit_cut,best_pv): 
    import math
    
    signal_sum    = 0
    cbkg_sum      = 0
    signal_sumw2  = 0
    signal_sum_wrong    = 0
    cbkg_sum_wrong       = 0
    signal_sumw2_wrong   = 0
    
    delta_m = 5065 
    S = 0.691 #sin2beta value (average from HFAG)
    
    timeres_power_bestPV = 0
    timeres_power_randomPV = 0
    

    
    tagomega = np.array(best_pv['B0_TAGOMEGA_OS'])
    tag1 = np.array(best_pv['B0_TAGDECISION_OS'])
    time = np.array(best_pv['B0_TAU']) 
    error = np.absolute(np.array(best_pv['Delta_TAU']))
    
    tagomega_wrong = np.array(data_fit_cut['B0_TAGOMEGA_OS'])
    tag1_wrong = np.array(data_fit_cut['B0_TAGDECISION_OS'])
    time_wrong = np.array(data_fit_cut['B0_TAU']) 
    error_wrong = np.absolute(np.array(data_fit_cut['Delta_TAU']))
    
    range_best_pv = best_pv.shape[0]-1
    print(range_best_pv)
    
    for i in range(range_best_pv):
        signal_weight =  1 
        cbkg_weight   =  1 
        mistag        =  tagomega[i] 
        tag           =  tag1[i]
        decaytime     =  time[i]
        timeerror     =  error[i]
        
        
        signal_sum    += signal_weight
        signal_sumw2  += signal_weight**2
        cbkg_sum      += cbkg_weight

        timeerror_dilution = math.exp(-(delta_m*timeerror)**2)

        timeres_power_bestPV    += timeerror_dilution*signal_weight
         
    print('+++++++++++++++++++++++++')
    print(cut)
    timeerror_dilution_theory = math.exp(-(delta_m*cut)**2)

    timeres_power_bestPV /= signal_sum
    #timeres_power_wrong /= signal_sum_wrong

    
    fomname_to_value = {
        'TimeresPower_bestPV' : timeres_power_bestPV,
        'TimeresTheory': timeerror_dilution_theory,
    }
    #return fomname_to_value

    for i in range(data_fit_cut.shape[0]):
        signal_weight_wrong =  1 
        cbkg_weight_wrong   =  1 
        mistag_wrong        =  tagomega_wrong[i] 
        tag_wrong           =  tag1_wrong[i]
        decaytime_wrong     =  time_wrong[i]
        timeerror_wrong     =  error_wrong[i]
        
        
        signal_sum_wrong    += signal_weight_wrong
        signal_sumw2_wrong  += signal_weight_wrong**2
        
        timeerror_dilution_wrong = math.exp(-(delta_m*timeerror_wrong)**2)
    
        timeres_power_randomPV     += timeerror_dilution_wrong *signal_weight_wrong 
        
    timeres_power_randomPV /= signal_sum_wrong  
    fomname_to_value.update({'TimeresPower_randomPV' : timeres_power_randomPV})
    
    return fomname_to_value

## Main loop

In [ ]:
from ROOT import RooArgList
import subprocess, os
import time

x=None
for cut in cuts:
    print('INFO: Starting to test cut ' + '{0}<{1}'.format(cuttype,cut), flush=True)
    data_fit_cut = signal_dataframe_wrongPV.query('{0}>-{1}&{0}<{1}'.format(cuttype,cut)) 
    best_pv = signal_dataframe_wrongPV.query('{0}>-{1}&{0}<{1}'.format(cuttype,cut)).query('idxPV==0') 
    #data_fit_cut = data_fit_cut.query('B0_FitPVConst_MinIPCHI2anyPV_flat>6')
    #print("The size of the sample is:")
    #print(data_fit_cut['Delta_TAU'].min())
    data_fit_cut.query('BDTresponse_wrongPV>4').to_root('/fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks/interim_bdtapproach.root',key=tree_mc)
    print('INFO: Call CandidateSelectionGrimReaper', flush=True)
    my_env = os.environ.copy()
    my_env['PATH'] = '/usr/local/bin:/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/sbin:/home/chasenberg/bin'
    my_env['LD_LIBRARY_PATH'] = '/usr/local/lib'
    my_env['script'] = '/home/chasenberg/repos/b2cc_sin2beta_run2/notebooks/selection'
    my_command = 'source /lhcbsoft/LHCbSoftwareSetup.sh &&' \
             'lb-run DaVinci/v41r2 $BASH -c "' \
             'echo $PATH && ' \
             'source /doosoft/InstallDooSoftware/LoadDooSoftware && CandidateSelectionGrimReaper /fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks/interim_bdtapproach.root Bd2JpsiKS /fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks/interim_bdtapproach_2.root Bd2JpsiKS "idxRunNumber""'                     
    subprocess.Popen([my_command], env=my_env, shell=True)
   # subprocess.call(('bash /home/chasenberg/repos/b2cc_sin2beta_run2/notebooks/selection/random_sel.sh'),shell=True)                      
    print("Wait for GrimReaper")
    time.sleep(40.0)    # pause 5.5 seconds
    print("GrimReaper hopefully finished")
    print('INFO: Read selected data', flush=True)
    data_fit_cut = rp.read_root('/fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks/interim_bdtapproach_2.root',key=tree_mc)
    print('INFO: Start Calculation', flush=True)
    #print('INFO: Building Ipatia (signal) + Exp (combinatorics) + Ipatia (bs-component) model for data', flush=True) 
    #final_model = build_mass_model(B0_FitDaughtersConst_M)
    
    cut_to_shape[cut] = {} 
    cut_to_performance[cut] = {}
   
    
    foms = calculate_foms(data_fit_cut,best_pv)
    cut_to_performance[cut].update(foms)
    
    print('INFO: Finish Calculation', flush=True)

INFO: Starting to test cut Delta_TAU<1e-05
INFO: Call CandidateSelectionGrimReaper
Wait for GrimReaper
GrimReaper hopefully finished
INFO: Read selected data
INFO: Start Calculation
94362
+++++++++++++++++++++++++
1e-05
INFO: Finish Calculation
INFO: Starting to test cut Delta_TAU<0.00011090909090909092
INFO: Call CandidateSelectionGrimReaper
Wait for GrimReaper
GrimReaper hopefully finished
INFO: Read selected data
INFO: Start Calculation
429367
+++++++++++++++++++++++++
0.000110909090909
INFO: Finish Calculation
INFO: Starting to test cut Delta_TAU<0.00021181818181818183
INFO: Call CandidateSelectionGrimReaper
Wait for GrimReaper
GrimReaper hopefully finished
INFO: Read selected data
INFO: Start Calculation
442271
+++++++++++++++++++++++++
0.000211818181818
INFO: Finish Calculation
INFO: Starting to test cut Delta_TAU<0.0003127272727272728
INFO: Call CandidateSelectionGrimReaper
Wait for GrimReaper
GrimReaper hopefully finished
INFO: Read selected data
INFO: Start Calculation
443467


In [ ]:
import numpy as np

def unpack_cut_dict(cut_dict, normalize=True):
    cuts = np.array(list(cut_dict.keys()))
    cuts.sort()
    
    new_dict = {}
    for cut in cuts:
        parameters_dict = cut_dict[cut]
        for parameter, value in parameters_dict.items():
            if parameter in new_dict:
                new_dict[parameter].append(value)
            else:
                new_dict[parameter] = [value]
    
    for parameter, values in new_dict.items():
        new_dict[parameter] = np.array(values)
        if normalize:
            new_dict[parameter] = values/np.max(np.abs(values))
    
    return cuts, new_dict

x, ys = unpack_cut_dict(cut_to_performance)
for parameter, values in ys.items():
    if "value" in parameter:
        print(values)
        y_errlo = np.array(ys[parameter.replace("_value","_errlo")])
        y_errhi = np.array(ys[parameter.replace("_value","_errhi")])
        y_errors= abs(np.vstack([y_errlo,y_errhi]))
        plt.errorbar(x,values,yerr=y_errors,marker='.',linestyle='', label='sigma')
        plt.xlabel(parameter)
        plotname = parameter + '.pdf'
        singleplot_file = os.path.join(plot_dir, plotname)
        plt.savefig(singleplot_file)
        plt.show()

### Plot FOMs

In [ ]:
import random
import seaborn as sns
sns.set_style("whitegrid")
sns.set_style("whitegrid", {"legend.frameon": True})

x, ys = unpack_cut_dict(cut_to_performance)
shift = 0.0

foms_to_plot = {}
#foms_to_plot["TaggingPower"] = r"Flavour Tagging"
foms_to_plot["TimeresPower_bestPV"] = "Best PV selection"
foms_to_plot["TimeresPower_randomPV"]   = "BDT correct PV selection"
#foms_to_plot["BsFOM"]        = "$X_i$-term"
#foms_to_plot["BdFOM_p"]      = "FOM ${}_{B^0_d}$"
#foms_to_plot["BdFOM"]        = "FOM"

#for parameter, values in ys.items():
for parameter in foms_to_plot:
    values=ys[parameter]
    #if "FOM" in parameter or "EffSig" in parameter or "s3_over_spb2" in parameter\
    #or "TaggingPower" in parameter or "TimeresPower" in parameter:
    if parameter in foms_to_plot:
        plt.errorbar(x,values+random.random()*shift,marker='.',linestyle='-',label=foms_to_plot[parameter])
        plt.xlabel("Delta_TAU cut")
'''values_EffSigSize=ys['EffSigSize']
values_TimeresPower=ys['TimeresPower']
values_TaggingPower=ys['TaggingPower']
values_BdFOM_p=ys['BdFOM_p']
values_BdFOM=ys['BdFOM']


plt.errorbar(x,values_EffSigSize+random.random()*shift,marker='.',linestyle='-',label=foms_to_plot[parameter],color=current_palette[0])
plt.errorbar(x,values_TimeresPower+random.random()*shift,marker='.',linestyle='-',label=foms_to_plot[parameter],color=current_palette[1])
plt.errorbar(x,values_TaggingPower+random.random()*shift,marker='.',linestyle='-',label=foms_to_plot[parameter],color=current_palette[2])
plt.errorbar(x,values_BdFOM_p+random.random()*shift,marker='.',linestyle='-',label=foms_to_plot[parameter],color=current_palette[3])
plt.errorbar(x,values_BdFOM+random.random()*shift,marker='.',linestyle='-',label=foms_to_plot[parameter],color=current_palette[4])
plt.xlabel("BDT Schnitt")'''
plt.gca().set_ylim(0.94,1.001)
plt.gca().set_xlim(0.0000000001,0.02)
plt.legend(loc='best')
plt.savefig(subdir + '/dilution_bdtapproach.pdf')
plt.show()

In [ ]:
print(ys['TimeresPower_bestPV'])

In [ ]:
print(ys['TimeresPower_randomPV'])

In [ ]:
print(x)

Plotting of B0_TAU and Delta_TAU

In [ ]:
delta_tau = RooRealVar("Delta_TAU", "delta_tau", 0,2, "ps")
b0_tauerr = RooRealVar("B0_TAUERR", "delta_tau", 0,2, "ps")

dataset_file_name =  '/fhgfs/users/chasenberg/mc/2015_2016_merged/jpsimumuks/interim.root'
data = ROOT.TFile(dataset_file_name)
tree_data = data.Get('Bd2JpsiKS') 
ntupleVarSet =  RooArgSet(delta_tau,b0_tauerr) 
dataset = RooDataSet('data','data',tree_data,ntupleVarSet)

In [ ]:
'''# Prepare frame
%matplotlib inline  
import ROOT

frame = delta_tau.frame(ROOT.RooFit.Bins(100))

dataset.plotOn(frame, ROOT.RooFit.Name("data1"))
text_size = 0.035
# Create TLegend
legend = ROOT.TLegend(0.7, 0.7,0.85,0.85)
legend.AddEntry(0,"LHCb unofficial", "")
legend.AddEntry(frame.findObject('data1'), "Delta_Tau", "p");
#legend.AddEntry(frame.findObject('sig_pdf_ext'), 'Massenfunktion', 'kBluw')

legend.SetTextSize(text_size)
frame.GetYaxis().SetTitle(frame.GetYaxis().GetTitle().replace("Events", "Kandidaten"))
# Plot pulls
#can, _ = plot_pulls('Hallo', frame, legend=legend, logy=False)#, latex=latex)
can = ROOT.TCanvas("my_canvas","Plotting Canvas",150,10,990,660)
can.SaveAs("/home/chasenberg/plots/wrongpv_cuttuning/delta_tau.pdf")
can  # To display plot in notebooks'''